In [1]:
import pandas as pd
import numpy as np

   首先我们最直接的就是查看我们右上角帮助里的pythonAPI文档， 里面的阐述的比较具体。
今天我们先用notebook实现这几点：
## 1 直接通过财务指标选取出来符合要求的股票（从单一到复合）
## 2 在已有的股票池中选取符合要求的股票
#### 我们先了解一下get_fundamentals()里的几个参数
   * query 是你要去查找的财务数据
   * entry_date 开始查找的日期，默认是NONE，在Notebook里是必须要填写的。在策略里是你回测的起始日期
   * interval 查询频率，比如说5天一查，5年一查
   * report_quarter 返回的时候是否返回是哪一个季度的


In [26]:
df = get_fundamentals(
    query(
        #在这里选取你要你要的财务指标，详情参考财务数据文档，我们先在这里按照市值大小来选取，market_cap
        fundamentals.eod_derivative_indicator.market_cap
    ),entry_date='2015-06-01',interval='5y'
)

In [27]:
df

<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 5 (major_axis) x 2748 (minor_axis)
Items axis: market_cap to market_cap
Major_axis axis: 2015-06-01 00:00:00 to 2011-05-30 00:00:00
Minor_axis axis: 300281.XSHE to 002574.XSHE

#### 注意这里是返回的是一个Panel,因为我们查询的是一个时间，股票代码，还有市值的一个三维的集，所以一定是会返回一个pannel那我们如果要返回一个dataFrame，那么我们就要减少一个维度，比如说，我们要的是300281.XSHE这一只股票

In [28]:
df.minor_xs('300281.XSHE')

,market_cap
2015-06-01,4.80118e+09
2014-05-30,1.5136e+09
2013-05-30,1.314e+09
2012-05-30,1.0758e+09
2011-05-30,NaN


In [36]:
stocks=np.array(df.minor_axis)
stocks

array(['300281.XSHE', '600089.XSHG', '600211.XSHG', ..., '300018.XSHE',
       '002692.XSHE', '002574.XSHE'], dtype=object)

#### 同理，我们要获取某一个时间的话就是用df.major_xs('2014-05-30')
这里，我们终于获得了API里面写的dataFrame了....即使很心塞，但是不得不承认完善的情况下是一定是三维的。
那么接下来我们通过财务数据最终是要选取股票出来，那么其实我们就是要获取到查询出来的这个panel里的minor_axis

这里我们使用numpy里的array来保存一下这个股票列表。（因为array保存的是相同的数据类型，list可以不是）
顺带一提我们经常在社区里看到的策略都有context.stocks = 一个DataFrame.columns.values，因为这是二维dataFrame获取的是列的值就是股票列表了

#### 下面我们直接来一个API里复合查询的demo：
   * filter方法是可以让你去筛选指标的，比如说demo里就是筛选pe大于55小于60之间
   * .order_by方法是一个排序的方法，demo里是用revenue来进行一个降序排列 desc()是降序，asc()是升序。
   * .limit方式是限制拿到数据的数量，因为最终你可能只需要排名靠前的10只

In [37]:
fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue, fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio > 55
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 60
        ).order_by(
            fundamentals.income_statement.revenue.desc()
        ).limit(
            10
        ),entry_date='2016-04-01',interval='5y'  #策略里填写如果只需要股票池，那么entry_date和interval是不需要填写的，但是notebook中是要的
    )

In [38]:
fundamental_df

<class 'pandas.core.panel.Panel'>
Dimensions: 2 (items) x 5 (major_axis) x 9 (minor_axis)
Items axis: revenue to pe_ratio
Major_axis axis: 2016-04-01 00:00:00 to 2012-03-30 00:00:00
Minor_axis axis: 600875.XSHG to 600297.XSHG

#### 这里依然是一个panel，那么我们先把股票池筛选出来,跟上面一样我们用一个array来存


In [42]:
stocks = np.array(fundamental_df.minor_axis)
stocks

array(['600875.XSHG', '601808.XSHG', '601727.XSHG', '600998.XSHG',
       '601866.XSHG', '600704.XSHG', '600011.XSHG', '000626.XSHE',
       '600297.XSHG'], dtype=object)

到这里，我们已经把第一步 从选财务数据中选出符合我们要求的股票完成了。
#### 接下来进行第二步，从我们已有的股票池里选取符合财务数据要求的。
   * 1.我们可以自己维护一个股票，比如在一开始定义context.stocks=['XXXXXX.XSHG',‘XXXX.XSHE’]你想要去查询的股票同样，这里我们直接选取某一个概念里的所有股票去找到里面符合我们财务数据要求的股票
      

In [46]:
stocks=concept('次新股')
stocks

['300427.XSHE',
 '300405.XSHE',
 '300404.XSHE',
 '300388.XSHE',
 '603315.XSHG',
 '002747.XSHE',
 '300398.XSHE',
 '603618.XSHG',
 '603015.XSHG',
 '002737.XSHE',
 '603012.XSHG',
 '603519.XSHG',
 '603609.XSHG',
 '300413.XSHE',
 '300429.XSHE',
 '300439.XSHE',
 '603009.XSHG',
 '603997.XSHG',
 '603158.XSHG',
 '603222.XSHG',
 '300386.XSHE',
 '603021.XSHG',
 '002728.XSHE',
 '603828.XSHG',
 '002750.XSHE',
 '300385.XSHE',
 '300428.XSHE',
 '603686.XSHG',
 '601016.XSHG',
 '300400.XSHE',
 '603199.XSHG',
 '000166.XSHE',
 '300397.XSHE',
 '603611.XSHG',
 '300423.XSHE',
 '603017.XSHG',
 '002726.XSHE',
 '603456.XSHG',
 '300399.XSHE',
 '603030.XSHG',
 '603306.XSHG',
 '300395.XSHE',
 '300437.XSHE',
 '002738.XSHE',
 '603939.XSHG',
 '300384.XSHE',
 '300433.XSHE',
 '603166.XSHG',
 '601689.XSHG',
 '002730.XSHE',
 '601198.XSHG',
 '603998.XSHG',
 '002742.XSHE',
 '300426.XSHE',
 '300424.XSHE',
 '603636.XSHG',
 '603567.XSHG',
 '603020.XSHG',
 '002749.XSHE',
 '603338.XSHG',
 '300415.XSHE',
 '601226.XSHG',
 '603799

 * 2.拿到之后我们就去查询：先解释一下不同的地方
    #### 我们一眼就看到关键的不同在于多了一个.filter()方法，我们是从具体指标的上一级中的方法.stockcode.in(参数)中查询我们传进去的那个股票列表
    #### 同样，这里查询的有两个指标，对应的上级也不一样，那么无论我们用fundamentals.eod_derivative_indicator.stockcode.in还是fundamentals.income_statement.stockcode.in返回的结果都一样

In [53]:
new_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio > 5
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 300
        ).filter(
            fundamentals.income_statement.stockcode.in_(stocks)
            ),entry_date='2015-04-01'
    )
stocks1=np.array(new_fundamental_df.minor_axis)
stocks1

array(['002724.XSHE', '002726.XSHE', '002727.XSHE', '002728.XSHE',
       '002729.XSHE', '002730.XSHE', '002731.XSHE', '002732.XSHE',
       '002733.XSHE', '002734.XSHE', '002735.XSHE', '002736.XSHE',
       '002737.XSHE', '002738.XSHE', '002739.XSHE', '002740.XSHE',
       '002741.XSHE', '002742.XSHE', '002743.XSHE', '002745.XSHE',
       '002746.XSHE', '002747.XSHE', '002748.XSHE', '002749.XSHE',
       '002750.XSHE', '300364.XSHE', '300374.XSHE', '300384.XSHE',
       '300385.XSHE', '300386.XSHE', '300387.XSHE', '300388.XSHE',
       '300389.XSHE', '300390.XSHE', '300391.XSHE', '300392.XSHE',
       '300393.XSHE', '300394.XSHE', '300395.XSHE', '300396.XSHE',
       '300398.XSHE', '300400.XSHE', '300401.XSHE', '300402.XSHE',
       '300403.XSHE', '300405.XSHE', '300406.XSHE', '300407.XSHE',
       '300408.XSHE', '300409.XSHE', '300410.XSHE', '300411.XSHE',
       '300412.XSHE', '300413.XSHE', '300415.XSHE', '300416.XSHE',
       '300417.XSHE', '300418.XSHE', '300419.XSHE', '300420.XS

### 呃，这里为了比较是不是真的成功了的话，我们可以再从整个市场里选取一下同样财务指标的股票，看看数量就知道是不是真的从我们自己定义的股票里选取了

In [66]:
old_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio > 5
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 300
        ),entry_date='2015-04-01'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

2091

# 搞完正经事情之后，我们来一些骚操作，比如说在.filter()加一些计算，或者是拿不同的指标进行比较！

In [67]:
old_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio +10  > 5
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < 300
        ),entry_date='2015-04-01'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

2093

#### 还真可以........其实用处不大，但是就是想试试。

In [73]:
old_fundamental_df = get_fundamentals(
        query(
            fundamentals.income_statement.revenue,      fundamentals.eod_derivative_indicator.pe_ratio
        ).filter(
            fundamentals.eod_derivative_indicator.pe_ratio < fundamentals.income_statement.revenue
        ),entry_date='2015-04-01'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

2272

In [85]:
import math
old_fundamental_df = get_fundamentals(
        query(
             fundamentals.eod_derivative_indicator.pb_ratio,     
        ).filter(
         fundamentals.eod_derivative_indicator.pb_ratio ** (1/4) >10000
        ),entry_date='2015-04-01'
    )
stocks2=np.array(old_fundamental_df.minor_axis)
len(stocks2)

TypeError: unsupported operand type(s) for ** or pow(): 'InstrumentedAttribute' and 'float'

### 实验证明，开个根号是不行的[微笑]，终于试出来了....